In [ ]:
"""
This code takes nearly 24 hours to complete.
Almost 2 and half hours for one model
"""

# import required packages

import shap
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import time

In [ ]:
def RunShap(infile, outfile, model_file):
    # time when started 
    st = time.time()
    
    # load model from the h5 file file 
    model = tf.keras.models.load_model(model_file)
    
    # load test data chunk
    testX = pd.read_csv(infile, sep=',', index_col=0)
    # droping last columns (labels)
    testX = testX.iloc[:, :-1] 
    testX = testX.to_numpy('float64')
        
    # run DeepExplaniner from shap pacage on the chunk 
    explainer_shap = shap.DeepExplainer(model=model, data=testX)
    shap_values = explainer_shap.shap_values(X=testX)
    np_shap = np.asarray(shap_values)
    # save shap values in a file 
    np.save(outfile, np_shap)
    
    # printing time required
    et = time.time()
    print('Time: ',et-st)


In [ ]:
# loop through 10 models
i = 1

while i < 11:
    
    # folder containing trained models and other data such as test data chunks 
    root_dir = '/path/to/models'
    
    # root_dir should have folder like DNN_1 , DNN_2 ... DNN_10  
    mname = "DNN_"+str(i)
    mdir = os.path.join(root_dir, mname)
    
    # every fold should h5 (HDF) model file 
    mfile = os.path.join(mdir, 'FPKM_5_MLP.h5')
    
    print('--- --- --- --- --- --- --- --- ---')
    print('Resdir:', mname)
    print('Model: ', mfile)
    
    # collect test data chunks from the same folder 
    # Our chunks file names starts with 'test' 
    # If you have changed the prefix please fix it in the code below
    tfiles = [f for f in os.listdir(mdir) if f.startswith('test')]
    tfiles.sort()

    for f in tfiles:        
        infile = os.path.join(mdir,f)
        outfile = os.path.join(mdir,"Shap_"+f)
        print(f)
        # shap interpretation on the chunk 
        RunShap(infile, outfile, mfile)

    i += 1
        
    print('--- --- --- --- ### --- --- --- ---')
